# Profitable App Profiles Guided Project

This project leverages app data to determine which kind of free apps would yield the most revenue through ads. We are preparing this for a imaginary company that produces free apps in English that is curious on the qualities of top free apps.

We will use two datasets from Kaggle to accomplish this:
1. [App Store](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)
2. [Google Play Store](https://www.kaggle.com/lava18/google-play-store-apps)

Sections below:

[0.0 Importing Data](#0.0-Importing-Data) <br>

[1.0 Data Cleaning](#1.0-Data-Cleaning) <br>
[1.1 Remove App Error](#1.1-Remove-App-Error) <br>
[1.2 Remove Duplicates](#1.2-Remove-Duplicates) <br>
[1.3 Isolate English Apps](#1.3-Isolate-English-Apps) <br>
[1.4 Isolate Free Apps](#1.4-Isolate-Free-Apps) <br>

[2.0 Analyze Apps](#2.0-Analyze-Apps) <br>
[2.1 Identifying Common App Genres](#2.1-Identifying-Common-App-Genres) <br>
[2.2 App User Numbers](#2.2-App-User-Numbers) <br>

## 0.0 Importing Data

In [12]:
# Open both datasets

from csv import reader

file = open('googleplaystore.csv', encoding='utf8')
read_file = reader(file)
android = list(read_file)
android_header = android[0]
android = android[1:]

file = open('AppleStore.csv', encoding='utf8')
read_file = reader(file)
apple = list(read_file)
apple_header = apple[0]
apple = apple[1:]

print('Total loaded apps: ' + str(len(android) + len(apple)))

Total loaded apps: 18038


In [13]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [14]:
explore_data(android, 0, 5, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Number of rows: 10841
Number of columns: 13


In [15]:
explore_data(apple, 0, 5, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 7197
Number of columns: 16


## 1.0 Data Cleaning

We will perform 4 steps to clean the data:
1. Remove one app with problematic data, as noted from the Kaggle dataset page
2. Remove duplicates present in the Google Play Store dataset (none are present in the App Store dataset)
3. Single out English apps only as these are the focus of this project
4. Finally, remove all non-free apps

### 1.1 Remove App Error

In [16]:
# It has been noted that the android dataset has an error in app index=10472
# This is android[10473] due to the header in android[0]
print('Correct Formatting:')
for col in range(len(android_header)):
    print(android_header[col], ': ', android[1][col])
print('\nIncorrect Formatting:')
for col in range(len(android_header)):
    print(android_header[col], ': ', android[10472][col])
# The data is off by one because the Category data is ommitted

Correct Formatting:
App :  Coloring book moana
Category :  ART_AND_DESIGN
Rating :  3.9
Reviews :  967
Size :  14M
Installs :  500,000+
Type :  Free
Price :  0
Content Rating :  Everyone
Genres :  Art & Design;Pretend Play
Last Updated :  January 15, 2018
Current Ver :  2.0.0
Android Ver :  4.0.3 and up

Incorrect Formatting:
App :  Life Made WI-Fi Touchscreen Photo Frame
Category :  1.9
Rating :  19
Reviews :  3.0M
Size :  1,000+
Installs :  Free
Type :  0
Price :  Everyone
Content Rating :  
Genres :  February 11, 2018
Last Updated :  1.0.19
Current Ver :  4.0 and up


IndexError: list index out of range

In [17]:
# Remove the problematic app data from android dataset
if android[10472][0].startswith('Life Made WI-Fi'):
    del android[10472]
print(android[10472])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


### 1.2 Remove Duplicates

In [18]:
# Next, remove duplicates that are present within the android dataset
# See example of duplicates below:
print(android_header)
for app in android:
    if app[0] == 'Instagram':
        print(app)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In [19]:
# How many such cases are there?
duplicates = 0
app_name = []
for app in android:
    if app[0] in app_name:
        duplicates += 1
    else:
        app_name.append(app[0])
print('Duplicates: ',duplicates)

Duplicates:  1181


In [20]:
# Remove duplicates by keeping only most recent entry (ie. with most ratings)
# The following method differs from the instruction of Dataquest, because I
# wanted to do it myself
# Start by making a dictionary that stores 'app name':[indices in android]
app_instances = {}
for i_app in range(len(android)):
    app_name = android[i_app][0]
    if app_name in app_instances:
        app_instances[app_name].append(i_app)
    else:
        app_instances[app_name] = [i_app]

header = 0
for i_app in app_instances:
    if header > 50:
        break
    if len(app_instances[i_app]) > 1:
        print('{} : {}'.format(app_instances[i_app],i_app))
        header += 1

[1, 2033] : Coloring book moana
[23, 2111] : Mcqueen Coloring pages
[36, 7027] : UNICORN - Color By Number & Pixel Art Coloring
[42, 4241] : Textgram - write on photos
[139, 4715] : Wattpad 📖 Free Books
[143, 4083] : Amazon Kindle
[145, 9621] : Dictionary - Merriam-Webster
[146, 6497] : NOOK: Read eBooks & Magazines
[155, 9642] : Oxford Dictionary of English : Free
[157, 9570] : Spanish English Translator
[162, 6509] : NOOK App for NOOK Devices
[164, 4095] : Ebook Reader
[186, 6296] : English Dictionary - Offline
[192, 292] : Docs To Go™ Free Office Suite
[193, 239, 268] : Google My Business
[194, 293] : OfficeSuite : Free Office + PDF Editor
[197, 7939] : Curriculum vitae App CV Builder Free Resume Maker
[201, 10200] : Facebook Pages Manager
[204, 236, 265] : Box
[206, 436] : Call Blocker
[213, 256] : ZOOM Cloud Meetings
[215, 10203] : Facebook Ads Manager
[222, 229, 285] : Quick PDF Scanner + OCR FREE
[228, 284] : SignEasy | Sign and Fill PDF and other Documents
[230, 286] : Genius S

In [21]:
# Helper to pick out android app row with the highest value in given column
# By default, this is the ratings in column index 3
def max_in_column(duplicates,column_index=3):
    column_vals = []
    for i_app in duplicates:
        column_vals.append(android[i_app][column_index])
    max_val = max(column_vals)
    return duplicates[column_vals.index(max_val)]

In [22]:
# Use app_instances and helper function to add each app to a new list called
# android_clean
print('Android app count: ', len(android))
android_clean = []
for app in app_instances:
    instances = app_instances[app]
    if len(instances) > 1:
        i_app_to_keep = max_in_column(instances)
        android_clean.append(android[i_app_to_keep])
    else:
        android_clean.append(android[instances[0]])
print('Android app count without duplicates: ', len(android_clean))

Android app count:  10840
Android app count without duplicates:  9659


### 1.3 Isolate English Apps

In [23]:
# Start by defining a function that evaluates whether a string is:
# - English only, most of the time (True)
# - Includes non-English (False)
def eng_check(s):
    strike = 0
    for c in s:
        if ord(c) > 127:
            strike += 1
        if strike == 4:
            return False
    return True
# Test the function on some examples
print(eng_check('Instagram'))
print(eng_check('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(eng_check('Docs To Go™ Free Office Suite'))
print(eng_check('Instachat 😜'))

True
False
True
True


In [24]:
# Apply the filter to create new so-called *_English datasets
def Englishify(ds,app_name_index):
    eng_ds = []
    for app in ds:
        if eng_check(app[app_name_index]):
            eng_ds.append(app)
    print('Non-English apps removed: ',len(ds)-len(eng_ds))
    return eng_ds
android_English = Englishify(android_clean,0)
apple_English = Englishify(apple,1)
print('Total remaining apps = {} + {} = {}'.format(len(android_English), \
                                                  len(apple_English), \
                                                  len(android_English) + len(apple_English)))

Non-English apps removed:  45
Non-English apps removed:  1014
Total remaining apps = 9614 + 6183 = 15797


### 1.4 Isolate Free Apps

In [25]:
# Check in each header where the price index is:
print(android_header) # index=7
print(apple_header)   # index=4

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [26]:
# Before pruning the data, look at the range of options here
def vals_in_col(ds,i):
    vals = []
    for app in ds:
        if app[i] not in vals:
            vals.append(app[i])
    print(vals)
vals_in_col(android_English,7)
vals_in_col(apple_English,4)
# We will have to strip away the '$' before checking the dollar-value

['0', '$4.99', '$3.99', '$6.99', '$1.49', '$2.99', '$7.99', '$5.99', '$3.49', '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49', '$10.00', '$24.99', '$11.99', '$79.99', '$16.99', '$14.99', '$1.00', '$29.99', '$12.99', '$2.49', '$10.99', '$1.50', '$19.99', '$15.99', '$33.99', '$74.99', '$39.99', '$3.95', '$4.49', '$1.70', '$8.99', '$2.00', '$3.88', '$25.99', '$399.99', '$17.99', '$400.00', '$3.02', '$1.76', '$4.84', '$4.77', '$1.61', '$2.50', '$1.59', '$6.49', '$1.29', '$5.00', '$13.99', '$299.99', '$379.99', '$37.99', '$18.99', '$389.99', '$19.90', '$8.49', '$1.75', '$14.00', '$4.85', '$46.99', '$109.99', '$154.99', '$3.08', '$2.59', '$4.80', '$1.96', '$19.40', '$3.90', '$4.59', '$15.46', '$3.04', '$4.29', '$2.60', '$3.28', '$4.60', '$28.99', '$2.95', '$2.90', '$1.97', '$200.00', '$89.99', '$2.56', '$30.99', '$3.61', '$394.99', '$1.26', '$1.20', '$1.04']
['0.0', '1.99', '0.99', '6.99', '2.99', '7.99', '4.99', '9.99', '3.99', '8.99', '5.99', '14.99', '13.99', '19.99', '17.99', '15.99

In [27]:
# Similar to the method in 1.3, go through each list to make another dataset
def get_free(ds,i_price):
    free_ds = []
    for app in ds:
        price = float(app[i_price].lstrip('$'))
        if price == 0.00:
            free_ds.append(app)
    print('Pay2Win apps removed: ',len(ds)-len(free_ds))
    return(free_ds)
android_free = get_free(android_English,7)
apple_free = get_free(apple_English,4)
print('Total remaining apps = {} + {} = {}'.format(len(android_free), \
                                                  len(apple_free), \
                                                  len(android_free) + len(apple_free)))

Pay2Win apps removed:  752
Pay2Win apps removed:  2961
Total remaining apps = 8862 + 3222 = 12084


## 2.0 Analyze Apps

Now the data is ready to be analyzed.

Since the business plan based on this analysis is to:
1. build an Android app available on Google Play,
2. develop it further if the app has a good response,
3. if app is profitable after 6 months, then build an iOS version for the App Store,

we will want to examine apps within our data that have performed well on both platforms.

### 2.1 Identifying Common App Genres

In [28]:
# First look at the columns we will need to assess in the datasets
print(android_header) # genre index=9, category index=1
print(apple_header)   # genre index=11
# two android dataset columns are indicative of the genre information

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [29]:
# Helper function to define a frequency table for a given column
def freq_table(ds, index):
    ds_len = len(ds)
    d = {}
    for app in ds:
        if app[index] in d:
            d[app[index]] += 100.0 / ds_len
        else:
            d[app[index]] = 100.0 / ds_len
    return d

# Helper function from Dataquest to sort the output of the frequency table
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [30]:
print('Category data')
display_table(android_free,1)

Category data
FAMILY : 18.9347777025507
GAME : 9.69307154141295
TOOLS : 8.451816745655742
BUSINESS : 4.592642744301517
LIFESTYLE : 3.904310539381615
PRODUCTIVITY : 3.893026404874732
FINANCE : 3.7011961182577164
MEDICAL : 3.5206499661475843
SPORTS : 3.3965244865718685
PERSONALIZATION : 3.3175355450236856
COMMUNICATION : 3.238546603475503
HEALTH_AND_FITNESS : 3.080568720379137
PHOTOGRAPHY : 2.945159106296538
NEWS_AND_MAGAZINES : 2.7984653577070557
SOCIAL : 2.6630557436244566
TRAVEL_AND_LOCAL : 2.335815842924842
SHOPPING : 2.245542766869776
BOOKS_AND_REFERENCE : 2.1439855563078267
DATING : 1.861882193635745
VIDEO_PLAYERS : 1.7941773865944455
MAPS_AND_NAVIGATION : 1.3992326788535314
FOOD_AND_DRINK : 1.2412547957571658
EDUCATION : 1.1735499887158662
ENTERTAINMENT : 0.959151433085084
LIBRARIES_AND_DEMO : 0.9365831640713173
AUTO_AND_VEHICLES : 0.9252990295644339
HOUSE_AND_HOME : 0.8237418190024836
WEATHER : 0.8011735499887168
EVENTS : 0.7109004739336499
PARENTING : 0.654479801399233
ART_AND_D

The above data is for the category column of the Google Play Store dataset. Family apps are the most common - those accessible to any age. Beyond this, Games and Tools and the most popular.

In [31]:
print('Genre data')
display_table(android_free,9)

Genre data
Tools : 8.440532611148859
Entertainment : 6.070864364703282
Education : 5.348679756262725
Business : 4.592642744301517
Productivity : 3.893026404874732
Lifestyle : 3.893026404874732
Finance : 3.7011961182577164
Medical : 3.5206499661475843
Sports : 3.464229293613168
Personalization : 3.3175355450236856
Communication : 3.238546603475503
Action : 3.1031369893929037
Health & Fitness : 3.080568720379137
Photography : 2.945159106296538
News & Magazines : 2.7984653577070557
Social : 2.6630557436244566
Travel & Local : 2.324531708417959
Shopping : 2.245542766869776
Books & Reference : 2.1439855563078267
Simulation : 2.0424283457458774
Dating : 1.861882193635745
Arcade : 1.8505980591288618
Video Players & Editors : 1.771609117580679
Casual : 1.7490408485669124
Maps & Navigation : 1.3992326788535314
Food & Drink : 1.2412547957571658
Puzzle : 1.1284134506883332
Racing : 0.9930038366057341
Role Playing : 0.9365831640713173
Libraries & Demo : 0.9365831640713173
Auto & Vehicles : 0.92529

For the genres of Google Play app dataset, Tools are the highest, even above entertainment apps. Surprisingly, leisure/games do not rank anywhere near the top here, but looking down below we can see that there are many subgenres for games that have divided that category into many genres: Action, Simulation, Arcave, Casual, Puzzle, Racing, Tole Playing, Strategy, etc.). Without adding all these up, we can see that the top genre games (action, simulation, arcade) equal 3.1% + 2% + 1.8% = 7% near to the percentage of the highest genre, that is Tools. Ultimately, it seems that family-friendly games are the most common.

In [32]:
display_table(apple_free,11)

Games : 58.1626319056464
Entertainment : 7.883302296710134
Photo & Video : 4.965859714463075
Education : 3.6623215394165176
Social Networking : 3.2898820608317867
Shopping : 2.6070763500931133
Utilities : 2.5139664804469306
Sports : 2.1415270018621997
Music : 2.048417132216017
Health & Fitness : 2.0173805090006227
Productivity : 1.7380509000620747
Lifestyle : 1.5828677839851035
News : 1.3345747982619496
Travel : 1.2414649286157668
Finance : 1.1173184357541899
Weather : 0.8690254500310364
Food & Drink : 0.8069522036002481
Reference : 0.558659217877095
Business : 0.5276225946617009
Book : 0.4345127250155184
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


For the App Store, there is clearly an overwhelming majority of game apps. Beyond this, later apps of different genres are in closer proportion, with 'Entertainment' apps coming in second. Generally speaking then, it can be said that popular App Store apps are for hobby and leisure rather than essential or practical purposes. If we are to produce a popular free English app, then a game seems best, though potentially that market is saturated. We would need to investigate other aspects of the data to get a better sense.

Overall, following out strategy, releasing a game on Google Play would probably work well on the App Store.

### 2.2 App User Numbers

Ideally we would have user counts available for each app, but this information is missing fro the App Store dataset. Instead, use the number of user ratings as a proxy for the App Store dataset.

In [33]:
genre_reviews = freq_table(apple_free, 11)
for genre in genre_reviews:
    reviews, genre_count = 0, 0
    for app in apple_free:
        if app[11] == genre:
            reviews += float(app[5])
            genre_count += 1
    print(genre, "average ratings:", reviews / genre_count)

Social Networking average ratings: 71548.34905660378
Photo & Video average ratings: 28441.54375
Games average ratings: 22788.6696905016
Music average ratings: 57326.530303030304
Reference average ratings: 74942.11111111111
Health & Fitness average ratings: 23298.015384615384
Weather average ratings: 52279.892857142855
Utilities average ratings: 18684.456790123455
Travel average ratings: 28243.8
Shopping average ratings: 26919.690476190477
News average ratings: 21248.023255813954
Navigation average ratings: 86090.33333333333
Lifestyle average ratings: 16485.764705882353
Entertainment average ratings: 14029.830708661417
Food & Drink average ratings: 33333.92307692308
Sports average ratings: 23008.898550724636
Book average ratings: 39758.5
Finance average ratings: 31467.944444444445
Education average ratings: 7003.983050847458
Productivity average ratings: 21028.410714285714
Business average ratings: 7491.117647058823
Catalogs average ratings: 4004.0
Medical average ratings: 612.0


As expected, the number of average ratings for games is rather low given how many game apps are present on the App Store. Navigation apps have the most ratings, followed by Reference apps, then Social Networking apps. Ideally we go for a type of app that has many reviews (ie. users) and is popular. A social networking app seems best then.

In [34]:
# Compare to installs for Google Play Store
display_table(android_free, 5)

1,000,000+ : 15.741367637102615
100,000+ : 11.55495373504876
10,000,000+ : 10.51681336041546
10,000+ : 10.200857594222716
1,000+ : 8.395396073121324
100+ : 6.91717445271956
5,000,000+ : 6.838185511171374
500,000+ : 5.574362446400399
50,000+ : 4.773188896411656
5,000+ : 4.513653802753331
10+ : 3.543218235161351
500+ : 3.249830737982386
50,000,000+ : 2.290679304897309
100,000,000+ : 2.12141728729406
50+ : 1.9183028661701613
5+ : 0.7898894154818334
1+ : 0.5077860528097492
500,000,000+ : 0.2708192281651996
1,000,000,000+ : 0.22568269013766634
0+ : 0.045136538027533285
0 : 0.011284134506883321


In [37]:
# Going forward, take all apps with x+ users to have contains x users exactly
# Now parse and compute an average like for the App Store data
category_users = freq_table(android_free, 1)
for category in category_users:
    users, category_count = 0, 0
    for app in android_free:
        if app[1] == category:
            users += float(app[5].replace(',','').replace('+',''))
            category_count += 1
    print(category, "average users:", users / category_count)

ART_AND_DESIGN average users: 1986335.0877192982
FAMILY average users: 3694276.334922527
AUTO_AND_VEHICLES average users: 647317.8170731707
BEAUTY average users: 513151.88679245283
BOOKS_AND_REFERENCE average users: 8767811.894736841
BUSINESS average users: 1712290.1474201474
COMICS average users: 817657.2727272727
COMMUNICATION average users: 38456119.167247385
DATING average users: 854028.8303030303
EDUCATION average users: 1820673.076923077
ENTERTAINMENT average users: 11640705.88235294
EVENTS average users: 253542.22222222222
FINANCE average users: 1387692.475609756
FOOD_AND_DRINK average users: 1924897.7363636363
HEALTH_AND_FITNESS average users: 4188821.9853479853
HOUSE_AND_HOME average users: 1331540.5616438356
TOOLS average users: 10682301.033377837
LIBRARIES_AND_DEMO average users: 638503.734939759
LIFESTYLE average users: 1437816.2687861272
GAME average users: 15560965.599534342
VIDEO_PLAYERS average users: 24727872.452830188
MEDICAL average users: 120616.48717948717
SOCIAL a